In [1]:
import os.path as osp
import subprocess
import os
from configs.constants import XMAGICAL_EMBODIMENT_TO_ENV_NAME

from absl import app
from absl import logging
from configs.constants import ALGORITHMS
from configs.constants import EMBODIMENTS
from torchkit.experiment import string_from_kwargs
from torchkit.experiment import unique_id
import yaml
import random


2024-11-06 21:44:05.938256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730929445.961063    3405 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730929445.968189    3405 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 21:44:05.992917: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
seeds = [0, 5]
device = "cuda:0"
# experiment_name = '/home/user/xirl/exp/exp602/'
experiment_name = '/home/user/xirl/exp/exp284'

with open(osp.join(experiment_name, "metadata.yaml"), "r") as fp:
    kwargs = yaml.load(fp, Loader=yaml.FullLoader)

if kwargs["algo"] == "goal_classifier":
    reward_type = "goal_classifier"
else:
    reward_type = "distance_to_goal"

# Map the embodiment to the x-MAGICAL env name.
env_name = XMAGICAL_EMBODIMENT_TO_ENV_NAME[kwargs["embodiment"]]

# Generate a unique experiment name.
# experiment_name = string_from_kwargs(
#   env_name=env_name,
#   reward="learned",
#   reward_type=reward_type,
#   mode=kwargs["mode"],
#   algo=kwargs["algo"],
#   uid=unique_id(),
# )
logging.info("Experiment name: %s", experiment_name)

# Execute each seed in parallel.
procs = []
for seed in range(*list(map(int, seeds))):
    procs.append(
        subprocess.Popen([  # pylint: disable=consider-using-with
            "python",
            "train_policy.py",
            "--experiment_name",
            experiment_name,
            "--env_name",
            f"{env_name}",
            "--config",
            f"configs/xmagical/rl/env_reward.py:{kwargs['embodiment']}",
            "--config.reward_wrapper.pretrained_path",
            f"{experiment_name}",
            "--config.reward_wrapper.type",
            f"{reward_type}",
            "--seed",
            f"{seed}",
            "--device",
            f"{device}",
            '--resume',
        ]))

# Wait for each seed to terminate.
for p in procs:
    p.wait()


FileNotFoundError: [Errno 2] No such file or directory: '/home/user/xirl/exp/exp284/metadata.yaml'